In [82]:
import numpy as np
import pandas as pd
from os import path, environ
import mrec

In [99]:
data = pd.read_csv('ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


импортируем датасет

In [100]:
data_dir = "/home/alex_k/Документы/mrec"
data = pd.read_csv('ml-100k/u.data', sep = '\t',header=None)
data.columns = ['item_id', 'movie_id', 'rating', 'timestamp']
data.head()

,item_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [101]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
item_id      100000 non-null int64
movie_id     100000 non-null int64
rating       100000 non-null int64
timestamp    100000 non-null int64
dtypes: int64(4)
memory usage: 3.1 MB


запустим многопоточность

In [18]:
! sbin/ipcluster start -n4 --daemonize

разделим датасет на обучающую и тестовую выборки

In [39]:
! sbin/mrec_prepare --dataset ml-100k/u.data --outdir splits  --test_size 0.2 --binarize

[2017-12-10 15:43:30,933] INFO: sorting input data...
[2017-12-10 15:43:31,396] INFO: creating split 0: /home/alex_k/Документы/mrec/splits/u.data.train.0 /home/alex_k/Документы/mrec/splits/u.data.test.0
[2017-12-10 15:43:32,367] INFO: creating split 1: /home/alex_k/Документы/mrec/splits/u.data.train.1 /home/alex_k/Документы/mrec/splits/u.data.test.1
[2017-12-10 15:43:33,418] INFO: creating split 2: /home/alex_k/Документы/mrec/splits/u.data.train.2 /home/alex_k/Документы/mrec/splits/u.data.test.2
[2017-12-10 15:43:34,285] INFO: creating split 3: /home/alex_k/Документы/mrec/splits/u.data.train.3 /home/alex_k/Документы/mrec/splits/u.data.test.3
[2017-12-10 15:43:35,190] INFO: creating split 4: /home/alex_k/Документы/mrec/splits/u.data.train.4 /home/alex_k/Документы/mrec/splits/u.data.test.4
[2017-12-10 15:43:36,065] INFO: cleaning up...
[2017-12-10 15:43:36,100] INFO: done


попробуем подобрать параметры регуляризации для модели slim

In [65]:
! sbin/mrec_tune\
    -d splits/u.data.train*\
    --input_format=tsv --l1_min=0.0001 --l1_max=1.0 --l2_min=0.00001 --l2_max=1.0 --max_sims=500 --min_sims=1 --max_sparse=0.3 

[2017-12-10 18:40:13,579] INFO: preparing tasks for a grid search of these values:
[2017-12-10 18:40:13,579] INFO: {'l1_reg': [0.0001, 0.001, 0.01, 0.1, 1], 'l2_reg': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1]}
[2017-12-10 18:40:13,949] INFO: running 30 tasks in parallel...
best parameter setting: {'l1_reg': 0.001, 'l2_reg': 1e-05}
mean # positive similarity weights per item = 2.31e+02
proportion of items with fewer than 1 positive similarity weights = 0.02
mean # negative similarity weights per item = 1.64e+02


## slim
обучим нашу модель с подобранными параметрами регуляризации

In [66]:
! sbin/mrec_train\
    -n 4\
    --input_format=tsv \
    --train='splits/u.data.train*' \
    --outdir=model_slim \
    --model=slim\
    --max_sims=500 \
    --l1_reg=0.001    \
    --l2_reg=0.00001 

[2017-12-10 18:43:26,200] INFO: processing /home/alex_k/Документы/mrec/splits/u.data.train.2...
[2017-12-10 18:43:26,200] INFO: finding number of items...
[2017-12-10 18:43:27,532] INFO: 943 users and 1682 items
[2017-12-10 18:43:27,533] INFO: creating sims directory /home/alex_k/Документы/mrec/model_slim/u.data.train.2-sims...
[2017-12-10 18:43:27,750] INFO: checking for existing output sims...
[2017-12-10 18:43:27,751] INFO: creating tasks...
[2017-12-10 18:43:27,752] INFO: running 4 tasks in parallel across ipython engines...
[2017-12-10 18:43:48,063] INFO: checking output files...
[2017-12-10 18:43:48,063] INFO: SUCCESS: all tasks completed
[2017-12-10 18:43:48,063] INFO: concatenating 4 partial output files...
[2017-12-10 18:43:48,181] INFO: removing partial output files...
[2017-12-10 18:43:48,551] INFO: loading 1682 items in SLIM model from /home/alex_k/Документы/mrec/model_slim/u.data.train.2.sims.tsv
[2017-12-10 18:43:56,857] INFO: done
[2017-12-10 18:43:56,857] INFO: processi

сделаем предсказание

In [67]:
! sbin/mrec_predict \
    --input_format=tsv\
    --test_input_format=tsv\
    --train='splits/u.data.train*'\
    --modeldir=model_slim\
    --outdir=model_slim_recs

[2017-12-10 18:46:37,481] INFO: processing /home/alex_k/Документы/mrec/splits/u.data.train.2...
[2017-12-10 18:46:37,482] INFO: creating recs directory /home/alex_k/Документы/mrec/model_slim_recs/u.data.train.2-recs...
[2017-12-10 18:46:37,618] INFO: checking for existing output recs...
[2017-12-10 18:46:37,620] INFO: creating tasks...
[2017-12-10 18:46:37,620] INFO: loading dataset to get size...
[2017-12-10 18:46:38,849] INFO: loading model to get size...
[2017-12-10 18:46:40,447] INFO: created 6 tasks, 158 users per task
[2017-12-10 18:46:40,447] INFO: running in parallel across ipython engines...
[2017-12-10 18:46:52,769] INFO: checking output files...
[2017-12-10 18:46:52,769] INFO: SUCCESS: all tasks completed
[2017-12-10 18:46:52,770] INFO: concatenating 6 partial output files...
[2017-12-10 18:46:52,859] INFO: removing partial output files...
[2017-12-10 18:46:52,860] INFO: done
[2017-12-10 18:46:52,863] INFO: processing /home/alex_k/Документы/mrec/splits/u.data.train.3...
[201

## KNN
обучим модель KNN, выберем k=300(в данной модели за этот параметр отвечает max_sims)

In [75]:
! sbin/mrec_train\
    -n 4\
    --input_format=tsv \
    --train='splits/u.data.train*' \
    --outdir=model_knn \
    --model=knn\
    --max_sims=300 \
    --metric=cosine

[2017-12-10 19:30:53,920] INFO: processing /home/alex_k/Документы/mrec/splits/u.data.train.2...
[2017-12-10 19:30:53,920] INFO: finding number of items...
[2017-12-10 19:30:55,151] INFO: 943 users and 1682 items
[2017-12-10 19:30:55,151] INFO: creating sims directory /home/alex_k/Документы/mrec/model_knn/u.data.train.2-sims...
[2017-12-10 19:30:55,158] INFO: checking for existing output sims...
[2017-12-10 19:30:55,159] INFO: creating tasks...
[2017-12-10 19:30:55,160] INFO: running 4 tasks in parallel across ipython engines...
[2017-12-10 19:31:11,037] INFO: checking output files...
[2017-12-10 19:31:11,038] INFO: SUCCESS: all tasks completed
[2017-12-10 19:31:11,039] INFO: concatenating 4 partial output files...
[2017-12-10 19:31:11,461] INFO: removing partial output files...
[2017-12-10 19:31:11,937] INFO: loading 1682 items in CosineKNNRecommender model from /home/alex_k/Документы/mrec/model_knn/u.data.train.2.sims.tsv
[2017-12-10 19:31:20,342] INFO: done
[2017-12-10 19:31:20,342] 

сделаем предсказание

In [76]:
! sbin/mrec_predict \
    --input_format=tsv\
    --test_input_format=tsv\
    --train='splits/u.data.train*'\
    --modeldir=model_knn\
    --outdir=model_knn_recs

[2017-12-10 19:33:59,905] INFO: processing /home/alex_k/Документы/mrec/splits/u.data.train.2...
[2017-12-10 19:33:59,905] INFO: creating recs directory /home/alex_k/Документы/mrec/model_knn_recs/u.data.train.2-recs...
[2017-12-10 19:33:59,990] INFO: checking for existing output recs...
[2017-12-10 19:33:59,992] INFO: creating tasks...
[2017-12-10 19:33:59,992] INFO: loading dataset to get size...
[2017-12-10 19:34:01,267] INFO: loading model to get size...
[2017-12-10 19:34:02,017] INFO: created 6 tasks, 166 users per task
[2017-12-10 19:34:02,017] INFO: running in parallel across ipython engines...
[2017-12-10 19:34:14,564] INFO: checking output files...
[2017-12-10 19:34:14,565] INFO: SUCCESS: all tasks completed
[2017-12-10 19:34:14,565] INFO: concatenating 6 partial output files...
[2017-12-10 19:34:14,594] INFO: removing partial output files...
[2017-12-10 19:34:14,601] INFO: done
[2017-12-10 19:34:14,605] INFO: processing /home/alex_k/Документы/mrec/splits/u.data.train.3...
[2017

## wrmf
обучим модель wrmf с заданными по умолчанию параметрами alpha и lbda, посмотрим на сколько хорошо работает данная модель при параметрах заданных по умолчанию

In [78]:
! sbin/mrec_train\
    -n 4\
    --input_format=tsv \
    --train='splits/u.data.train*' \
    --outdir=model_wrmf \
    --model=wrmf\
    --alpha=1.0 

[2017-12-10 19:50:17,380] INFO: processing /home/alex_k/Документы/mrec/splits/u.data.train.2...
[2017-12-10 19:50:17,381] INFO: creating factors directory /home/alex_k/Документы/mrec/model_wrmf/u.data.train.2-factors
[2017-12-10 19:50:17,455] INFO: getting data size
[2017-12-10 19:50:18,852] INFO: iteration 0
[2017-12-10 19:50:30,749] INFO: iteration 1
[2017-12-10 19:50:42,414] INFO: iteration 2
[2017-12-10 19:50:54,088] INFO: iteration 3
[2017-12-10 19:51:06,399] INFO: iteration 4
[2017-12-10 19:51:19,865] INFO: iteration 5
[2017-12-10 19:51:33,090] INFO: iteration 6
[2017-12-10 19:51:46,028] INFO: iteration 7
[2017-12-10 19:51:58,923] INFO: iteration 8
[2017-12-10 19:52:11,707] INFO: iteration 9
[2017-12-10 19:52:25,135] INFO: iteration 10
[2017-12-10 19:52:42,157] INFO: iteration 11
[2017-12-10 19:52:56,429] INFO: iteration 12
[2017-12-10 19:53:10,944] INFO: iteration 13
[2017-12-10 19:53:26,716] INFO: iteration 14
[2017-12-10 19:53:45,377] INFO: removing partial output files
[2017-

делаем предсказание

In [79]:
! sbin/mrec_predict \
    --input_format=tsv\
    --test_input_format=tsv\
    --train='splits/u.data.train*'\
    --modeldir=model_wrmf\
    --outdir=model_wrmf_recs

[2017-12-10 20:08:22,824] INFO: processing /home/alex_k/Документы/mrec/splits/u.data.train.2...
[2017-12-10 20:08:22,824] INFO: creating recs directory /home/alex_k/Документы/mrec/model_wrmf_recs/u.data.train.2-recs...
[2017-12-10 20:08:22,916] INFO: checking for existing output recs...
[2017-12-10 20:08:22,917] INFO: creating tasks...
[2017-12-10 20:08:22,917] INFO: loading dataset to get size...
[2017-12-10 20:08:24,975] INFO: loading model to get size...
[2017-12-10 20:08:25,164] INFO: created 1 tasks, 1165 users per task
[2017-12-10 20:08:25,164] INFO: running in parallel across ipython engines...
[2017-12-10 20:08:31,017] INFO: checking output files...
[2017-12-10 20:08:31,017] INFO: SUCCESS: all tasks completed
[2017-12-10 20:08:31,018] INFO: concatenating 1 partial output files...
[2017-12-10 20:08:31,104] INFO: removing partial output files...
[2017-12-10 20:08:31,105] INFO: done
[2017-12-10 20:08:31,107] INFO: processing /home/alex_k/Документы/mrec/splits/u.data.train.3...
[20

In [223]:
pred = pd.read_csv('model_slim_recs/u.data.train.1.recs.tsv',sep = '\t',header = None,index_col=None)
pred.columns = ['item_id', 'movie_id', 'pred']
pred.drop(['item_id'],axis=1,inplace=True)
item = pd.read_csv('ml-100k/u.item', sep = '|',header = None,encoding='ISO-8859-1')
item.drop([2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],axis=1,inplace=True)
item.columns = ['movie_id', 'title']
item.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [224]:
merg = pred.merge( item, how = 'inner', left_on = 'movie_id', right_on = 'movie_id' )
merg.head()
merg.sort_values('pred')[::-1].head(10)

,movie_id,pred,title
1736,100,2.572532,Fargo (1996)
6593,269,2.567473,"Full Monty, The (1997)"
4608,50,2.549860,Star Wars (1977)
12003,294,2.488853,Liar Liar (1997)
6775,7,2.435576,Twelve Monkeys (1995)
12269,22,2.433006,Braveheart (1995)
11210,286,2.352448,"English Patient, The (1996)"
79,258,2.350323,Contact (1997)
3597,181,2.349335,Return of the Jedi (1983)
2233,257,2.347752,Men in Black (1997)
